### Daten importieren

In [ ]:
import pandas as pd
import plotly.express as px
import numpy as np

### Daten auslesen

In [ ]:
df = pd.read_csv('flower_data.csv')

dict = {'name': 'Name',
        'height (cm)': 'Höhe',
        'longevity (years)': 'Lebenserwartung',
        'place of origin': 'Herkunftskontinent',
        'color': 'Farbe',
        'climate': 'Klima',
        'cut flowers': 'Geschnittene Blume',
        'perfumes': 'Parfum',
        'medicine': 'Medizin',
        'average number of petals': 'Durchschnittliche Anzhal Blätter'}

df.rename(columns=dict, inplace=True)

display(df)

### Daten filtern

In [ ]:
# Split the "Herkunftskontinent" column
df["Herkunftskontinent"] = df["Herkunftskontinent"].str.split(", ")

# Explode the DataFrame
df = df.explode("Herkunftskontinent")

# Filter the DataFrame for "Asia" and "Europe"
df_filtered = df.loc[df["Herkunftskontinent"].isin(["Asia", "Europe"])]

# Print the resulting DataFrame
df_filtered.head()

### Kreis-Diagramm: Herkunftskontinent

In [ ]:
font_family = 'Arial'

df["Herkunftskontinent"] = df["Herkunftskontinent"].str.split(", ")
df = df.explode("Herkunftskontinent")

#counts all instances of continents
origin_counts = df["Herkunftskontinent"].value_counts()

#plotly pastell color palette
colors = px.colors.qualitative.Pastel

#creates chart
fig = px.pie(values=origin_counts, names=origin_counts.index, title="Herkunftskontinent der Blumen", hole=0.5,
             color_discrete_sequence=colors)

#layout adjustments
fig.update_layout(title={'font': {'family': font_family, 'size': 24}, 'x': 0.5, 'y': 0.9},
                  font={'family': font_family, 'size': 12},
                  legend_title='Kontinente',
                  legend={'x': 0.75,  'y': 0.5})

fig.show()

### Kuchen-Diagramm: Europa vs Asien

In [ ]:
font_family = 'Arial'

asia_europe = df_filtered["Herkunftskontinent"].value_counts()

#plotly pastell color palette
colors = px.colors.qualitative.Pastel

#creates pie chart form asia europe series
fig = px.pie(values=asia_europe, names=asia_europe.index, title="Asia vs Europe")
                                                                                      
#add dropdown

#layout adjustments
fig.update_layout(
    title={'font': {'family': font_family, 'size': 24}, 'x': 0.5, 'y': 0.9},
    font={'family': font_family, 'size': 12},
    legend_title='Gewählte Kontinente',
    legend={'x': 0.7,  'y': 0.5}
    )

fig.show()

In [ ]:
### Aufgabe 5a) and 5b)

In [ ]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# generate random normal distributed data for x and y
# and store it in a pandas DataFrame

df = pd.DataFrame({'y': np.random.normal(loc=0, scale=10, size=1000),
                   'x': np.random.normal(loc=10, scale=2, size=1000)})

app.layout = html.Div([html.H1("Dashboard 2"),
    dbc.Row([
        
        dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color', multi=False)], width=6),

        ## Aufgabe 5a
        dbc.Col([dcc.RangeSlider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), value=[math.floor(df['y'].min()), math.ceil(df['y'].max())], id="my-range-slider")], width=6)
        ]),

    ## Aufgabe 5b
    dbc.Row([
        dbc.Col([dcc.Slider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), id="min_value")], width=6),
        dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color_2', multi=False)], width=6),
    ]),

    dbc.Row([dbc.Col([dcc.Graph(id="graph_1")], width=6),
             dbc.Col([dcc.Graph(id="graph_2")], width=6)
    ])], className="m-4")

@app.callback(Output("graph_1", "figure"), Input("color", "value"), Input("min_value", "value"))

def update_graph_1(dropdown_value_color, min_value):
    dff = df[df['y']> min_value]
    fig = px.histogram(dff, x="y", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout()
    return fig


## Aufgabe 5a und 5b
@app.callback(Output("graph_2", "figure"), [Input("my-range-slider", "value"), Input("color_2", "value")])

def update_graph_2(range_values, color):
    dff = df[(df['y'] > range_values[0]) & (df['y'] < range_values[1])]
    fig = px.scatter(dff, x='x', y='y', color_discrete_sequence=[color])
    fig.update_layout()
    return fig


if __name__ == '__main__':
    app.run_server(debug=False, port=8000)

In [ ]:
### Aufgabe 6 a) und b)

In [ ]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Random Daten für Tab 1
df_random = pd.DataFrame({'y': np.random.normal(loc=0, scale=10, size=1000),
                          'x': np.random.normal(loc=10, scale=2, size=1000)})

# Unser Datensatz --> für Streudiagrmam aus eigenem Dashboard (graph_3)
df_own = pd.read_csv('flower_data.csv')

dict = {'name': 'Name',
        'height (cm)': 'Höhe',
        'longevity (years)': 'Lebenserwartung',
        'place of origin': 'Herkunftskontinent',
        'color': 'Farbe',
        'climate': 'Klima',
        'cut flowers': 'Geschnittene Blume',
        'perfumes': 'Parfum',
        'medicine': 'Medizin',
        'average number of petals': 'Durchschnittliche Anzhal Blätter'}

df_own.rename(columns=dict, inplace=True)


app.layout = html.Div([
    html.Div([html.H1("Dashboard 3")], className="header"),
    html.Div([
        dcc.Tabs(id="tabs", children=[

            # Tab 1 (vorgegeben)
            dcc.Tab(label='Tab One', id="tab_1_graphs", children=[
                html.Div([
                    dbc.Row([
                        dbc.Col([dcc.Dropdown(options=['red', 'green', 'blue'], value='red', id='color', multi=False)], width=6),
                        dbc.Col([dcc.Slider(min=math.floor(df['y'].min()), max=math.ceil(df['y'].max()), id="min_value")], width=6)
                    ]),
                    dbc.Row([
                        dbc.Col([dcc.Graph(id="graph_1")], width=6),
                        dbc.Col([dcc.Graph(id="graph_2")], width=6)
                    ])
                ], className="tab_content")
            ]),

            # Tab 2 für Aufgabe 6a
            dcc.Tab(label='Tab Two', id="tab_2_graphs", children=[
    html.Div([
                dbc.Row([
                    dbc.Col([dcc.Graph(id="graph_4")], width=6)
        ])
    ])
]),

            # Tab 3 für Aufgabe 6b
            dcc.Tab(label='Tab Three', id="tab_3_graphs", children=[
                html.Div([
                    dbc.Row([
                        dbc.Col([dcc.Graph(id="graph_5")], width=6),
                    ])
                ])
            ])
        ])
    ])
])


@app.callback(Output("graph_1", "figure"), Input("color", "value"))
def update_graph_1(dropdown_value_color):
    fig = px.histogram(df, x="y", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout(template="plotly_white")
    return fig


@app.callback(Output("graph_2", "figure"), Input("min_value", "value"))
def update_graph_2(min_value):
    if min_value:
        dff = df[df['y'] > min_value]
    else:
        dff = df
    fig = px.scatter(dff, x='x', y='y')
    fig.update_layout(template="plotly_white")
    return fig

# callback für Aufgabe 6a (graph_3)
@app.callback(Output("graph_3", "figure"), Input("min_value", "value")) 
def update_graph_3(dropdown_value_color):
    fig = px.histogram(df, x="Durchschnittliche Anzhal Blätter", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout(template="plotly_white")
    return fig

# callback für Aufgabe 6b (graph_5)
@app.callback(Output("graph_5", "figure"), Input("min_value", "value"))
def update_graph_5(min_value):
    if min_value:
        dff = df_own[df_own['Höhe'] > min_value]
    else:
        dff = df_own
    fig = px.scatter(dff, x='Durchschnittliche Anzhal Blätter', y="Höhe")
    return fig

# callback für Balkendiagramm in Tab Two, Aufgabe 6a
@app.callback(Output("graph_4", "figure"), Input("min_value", "value"))
def update_graph_4(min_value):
    if min_value:
        dff = df_own[df_own['Höhe'] > min_value]
    else:
        dff = df_own
    
    # Daten für das Balkendiagramm
    count_by_color = dff['Farbe'].value_counts()
    colors = count_by_color.index.tolist()
    counts = count_by_color.tolist()
    
    # Erstellen des Balkendiagramms
    fig = px.bar(x=colors, y=counts)
    fig.update_layout(template="plotly_white")
    fig.update_xaxes(title="Farbe")
    fig.update_yaxes(title="Anzahl der Blumen")
    
    return fig

if __name__ == '__main__':
    app.run_server(debug=False, port=8010)

In [ ]:
## Testing

In [55]:
import math
from dash import Dash, dcc, html, Input, Output
import plotly.express as px
import numpy as np
import pandas as pd
import dash_bootstrap_components as dbc

# new: plotly template="plotly_white", https://plotly.com/python/templates/

app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

df = pd.DataFrame({'y': np.random.normal(loc=0, scale=10, size=1000), 'x': np.random.normal(loc=10, scale=2, size=1000)})

app.layout = html.Div([html.Div([html.H1("Dashboard 3")], className="header"), html.Div([dcc.Tabs(id="tabs", children=[
            dcc.Tab(label='Tab One', id="tab_1_graphs", children=[html.Div([
                dbc.Row([dbc.Col([dcc.Dropdown(options=['red','green','blue'], value='red', id='color', multi=False)], width=6),
                         dbc.Col([dcc.Slider(min=math.floor(df['y'].min()),max=math.ceil(df['y'].max()),id="min_value")], width=6)]),
                dbc.Row([dbc.Col([dcc.Graph(id="graph_1")],width=6),
                         dbc.Col([dcc.Graph(id="graph_2")],width=6)])], className="tab_content"),]),
            dcc.Tab(label='Tab Two', id="tab_2_graphs",children=[html.Div([],className="tab_content")]),

            #adding athird tab
            dcc.Tab(label='Tab Three', id="tab_3_graphs",children=[html.Div([],className="tab_content")]),])], className="content")])

@app.callback(Output("graph_1", "figure"), Input("color", "value"))

def update_graph_1(dropdown_value_color):
    fig = px.histogram(df, x="y", color_discrete_sequence=[dropdown_value_color])
    fig.update_layout(template="plotly_white")
    return fig

@app.callback(Output("graph_2", "figure"),Input("min_value", "value"))

def update_graph_2(min_value):
    if min_value:
        dff = df[df['y'] > min_value]
    else:
        dff = df
    fig = px.scatter(dff, x='x', y='y')
    fig.update_layout(template="plotly_white")
    return fig

if __name__ == '__main__':
    app.run_server(debug=False, port=8000)

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is running on http://127.0.0.1:8000/

Dash is run